In [1]:
from langchain_community.document_loaders import PyPDFLoader 

# PDF Based Document Loader
pdf_loader = PyPDFLoader(r'docs/key-fact-statement.pdf')
docs = pdf_loader.load()
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter, HTMLHeaderTextSplitter, RecursiveJsonSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50 )
# divide document into 500 text size and overalap of 50 characters

final_docs = text_splitter.split_documents(docs)

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(final_docs, embeddings)

In [10]:
query = "APR for Visa Gold"
retrieved_result = db.similarity_search(query)

In [11]:
retrieved_result

[Document(metadata={'author': 'jason.c.f.chow@hsbc.com.hk', 'creationdate': '2024-10-18T14:40:05+08:00', 'creator': 'Microsoft® Word for Microsoft 365', 'keywords': 'RESTRICTED -', 'moddate': '2024-10-18T14:40:05+08:00', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_actionid': '8c75c534-6cc7-4fbb-8718-2dcd08b8899b', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_contentbits': '2', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_enabled': 'true', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_method': 'Privileged', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_name': 'CLAPUBLIC', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_setdate': '2024-10-08T15:56:53Z', 'msip_label_3486a02c-2dfb-4efe-823f-aa2d1f0e6ab7_siteid': 'e0fd434d-ba64-497b-90d2-859c472e1a92', 'page': 3, 'page_label': '4', 'producer': 'Microsoft® Word for Microsoft 365', 'source': 'docs/key-fact-statement.pdf', 'subject': 'Key Fact Statement', 'title': 'Key Fact Statement', 'total_pages': 9}, page_content='Pract